<a href="https://colab.research.google.com/github/gustavoalcantara/capim/blob/main/capim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Case Data Analyst - Capim**
##**Gustavo Alcântara**
###**São José dos Campos, SP, Brasil**
###**Foco: Compreender o período de trial e como ele pode ser melhorado.**

Este Case faz parte da etapa do processo seletivo para a vaga de Data Anaylist da [Capim](https://capim.com.br/site). Todos os códigos/gráficos podem ser acessados via este colab e, caso necessário, replicados em seu ambiente. Verifique a necessidade dos arquivos utilizados estarem em seu Local.


#####Importe as bibliotecas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

#####Leia os arquivos localmente

In [3]:
#Reading the csvs archives
activity = pd.read_csv('activity.csv')
clinics = pd.read_csv('clinics.csv')
subscriptions = pd.read_csv('subscriptions.csv')

##Compreensão das tabelas e variáveis

####Contexto
A tabela `activity` possui 12 colunas, tendo um total de 13941 observações. Neste contexto, essa tabela traz o Id de cada clinica atendida no período de Agosto a Setembro de 2023 e a interação de cada uma das clinicas com as variáveis. Ao total, são `1070` clinicas diferentes que estão nesse dataframe. O arquivo `clinics` possui 2219 observações que contempla um total de 2219 clinicas diferentes e a tabela `subscriptions` possui 389 clínicas. Portanto, ao realizar uma junção interna (inner join) entre essas duas ou mais tabelas no futuro, apenas as clínicas presentes em ambos os conjuntos de dados serão consideradas.

#####Principal tipo de atividade
Pensando em `um Dashboard operacional para o time de Produto - Aquisição acompanhar este processo, com gráficos e números gerais a serem monitorados diariamente e semanalmente` optei por apresentar um gráfico da interação das clinicas com a plataforma a partir da variável `ACTIVITY_TYPE` no período agosto a setembro de 2023. É possível observar que as atividades que mais ocorrem é a de `criação de pacientes` e `procedimentos criados`. É válido refletir que a plataforma, de fato, tem sua utilização para monitoramento e SaaS para gestão odontólógica.

In [ ]:
(activity['ACTIVITY_TYPE'].value_counts()
 .reset_index()
 .pipe((px.bar, 'data_frame'), x='index', y='ACTIVITY_TYPE',
       labels={'index':'Tipo de Atividade', 'ACTIVITY_TYPE':'Contagem'},
       title='Contagem de Cada Tipo de Atividade no periodo de Ago. a Set. 2023'))

Mas, é necessário ter em mente como o periódo de trial pode ser melhorado. Diante disso, optei por verificar se as clinicas `Independente`, `Independente - Legado` e `Rede Homologada` possuem diferenças na duração de tempo de Trial. É fato que a diferença da quantidade de clinicas em cada categoria pode interferir neste valor. Porém, é perceptivel que o número de clinicas Indepentes é o que possui a maior média de tempo de Trial. Será que essas clínicas possuem caracteristicas e interesses diferentes na utilização da plataforma?

In [ ]:
(y.groupby('BUSINESS_SEGMENTATION')
   .agg({'CLINIC_ID':'count', 'TRIAL_DURATION':'mean'})
   .rename(columns={'CLINIC_ID':'Número de Clínicas',
                    'TRIAL_DURATION':
                    'Duração Média do Trial'}))

,Número de Clínicas,Duração Média do Trial
BUSINESS_SEGMENTATION,,
Independente,2055,8.474453
Independente - Legado,21,8.428571
Rede Homologada,143,7.146853


Assim, realizei uma análise dos interesses de cada atividade por segmentação de negócio. É possível observar que as `Redes Independentes` são as que tem maior interesse em utilizar o `BNPL`. Tendo isso em mente, é válido analisar essas clínicas no contexto da utilização da plataforma e sua interação com o período de Trial.  


In [13]:
result = (clinics.groupby(['INTEREST_CATEGORY_SIGNUP', 'BUSINESS_SEGMENTATION'])
          .agg(qtde_clinicas=('CLINIC_ID', 'nunique'))
          .reset_index()
          .sort_values(by='qtde_clinicas', ascending=False)
          .head(10))

# Criar o gráfico de barras empilhadas com Plotly Express
fig = (px.bar(result,
             x='INTEREST_CATEGORY_SIGNUP',
             y='qtde_clinicas',
             color='BUSINESS_SEGMENTATION',
             title="Contagem de Clínicas por Interesse e Segmentação de Negócios",
             labels={'INTEREST_CATEGORY_SIGNUP': 'Interesses', 'qtde_clinicas': 'Número de Clínicas'},
             text='qtde_clinicas')
       .update_xaxes(tickangle=45, tickmode='array'))

# Mostrar o gráfico
fig.show()


Tendo em vista essa situação, o que pode influenciar uma clínica Independente contemplar ou não a assinatura? Vamos analisar quais os tipos de atividades realizas por essas clínicas que fizeram ou não a assinatura. Assim, no período de análise, 241 clinícas estão com a assinatura `active`, 24 estão com o status de `canceled` e `incomplete` e `past_due` somam-se a 6 clinicas. Atente-se ao `INNER JOIN` como parametro aqui.

In [17]:
(activity.merge(clinics, on='CLINIC_ID')
    .merge(subscriptions, on='CLINIC_ID')
    .query("BUSINESS_SEGMENTATION == 'Independente'")
    .groupby('SUBSCRIPTION_STATUS')
    .agg(count=('CLINIC_ID', 'nunique'))
    .reset_index()
)

,SUBSCRIPTION_STATUS,count
0,active,241
1,canceled,24
2,incomplete,1
3,past_due,5


Dessas 24 clínicas que cancelaram o Trial, existe o seguinte interesse em comum:

In [21]:
(activity.merge(clinics, on='CLINIC_ID')
    .merge(subscriptions, on='CLINIC_ID')
    .query("BUSINESS_SEGMENTATION == 'Independente'")
    .groupby(['SUBSCRIPTION_STATUS', 'INTEREST_CATEGORY_SIGNUP'])
    .agg(count=('CLINIC_ID', 'nunique'))
    .query('SUBSCRIPTION_STATUS == "canceled"')
    .reset_index()
)

,SUBSCRIPTION_STATUS,INTEREST_CATEGORY_SIGNUP,count
0,canceled,Apenas BNPL,9
1,canceled,Apenas SaaS,2
2,canceled,BNPL e SaaS,13


Sendo assim, as que possuem interesse em apenas o BNPL interagiram com a plataforma da seguinte maneira:

In [22]:
(activity.merge(clinics, on='CLINIC_ID')
    .merge(subscriptions, on='CLINIC_ID')
    .query("BUSINESS_SEGMENTATION == 'Independente'")
    .groupby(['SUBSCRIPTION_STATUS', 'ACTIVITY_TYPE', 'INTEREST_CATEGORY_SIGNUP'])
    .agg(count=('CLINIC_ID', 'nunique'))
    .reset_index()
    .query('SUBSCRIPTION_STATUS == "canceled" and INTEREST_CATEGORY_SIGNUP == "Apenas BNPL"')
    .sort_values(by='count', ascending=False)
)

,SUBSCRIPTION_STATUS,ACTIVITY_TYPE,INTEREST_CATEGORY_SIGNUP,count
85,canceled,pre_analysis,Apenas BNPL,6
70,canceled,budget_created,Apenas BNPL,3
79,canceled,patient_created,Apenas BNPL,3
87,canceled,procedure_created,Apenas BNPL,3
96,canceled,time_slot_created,Apenas BNPL,3
66,canceled,anamnesis_created,Apenas BNPL,2
74,canceled,document_created,Apenas BNPL,1
75,canceled,folder_created,Apenas BNPL,1
77,canceled,new_user_created,Apenas BNPL,1
82,canceled,patient_discarded,Apenas BNPL,1


É possível observar que as clinicas que cancelaram a sua assinatura tem um comportamento de `pre_analysis` para seu paciente. O que difere, portanto, essas clínicas das que possuem essa caracteristica de cancelamento com as que não cancelaram? Vamos observar o gráfico a seguir:

In [25]:
def generate_activity_plot(data, title):
    fig = px.bar(data, x='ACTIVITY_TYPE', y='count', title=title)
    fig.show()

cancelled_data = (activity
    .merge(clinics, on='CLINIC_ID')
    .merge(subscriptions, on='CLINIC_ID')
    .query("BUSINESS_SEGMENTATION == 'Independente'")
    .groupby(['SUBSCRIPTION_STATUS', 'ACTIVITY_TYPE', 'INTEREST_CATEGORY_SIGNUP'])
    .agg(count=('CLINIC_ID', 'nunique'))
    .reset_index()
    .query('SUBSCRIPTION_STATUS == "canceled" and INTEREST_CATEGORY_SIGNUP == "Apenas BNPL"')
    .sort_values(by='count', ascending=False)
)

not_cancelled_data = (activity
    .merge(clinics, on='CLINIC_ID')
    .merge(subscriptions, on='CLINIC_ID')
    .query("BUSINESS_SEGMENTATION == 'Independente'")
    .groupby(['SUBSCRIPTION_STATUS', 'ACTIVITY_TYPE', 'INTEREST_CATEGORY_SIGNUP'])
    .agg(count=('CLINIC_ID', 'nunique'))
    .reset_index()
    .query('SUBSCRIPTION_STATUS == "active" and INTEREST_CATEGORY_SIGNUP == "Apenas BNPL"')
    .sort_values(by='count', ascending=False)
)
generate_activity_plot(cancelled_data, 'Atividades para Usuários que Cancelaram')
generate_activity_plot(not_cancelled_data, 'Atividades para Usuários que Não Cancelaram')


É possível observar que a partir do `pre_analysis` já existe a criação de pacientes para as clínicas que não cancelaram. Será que é falta de expertise com a plataforma? Tendo o feedback dos usuários para com a plataforma, pode ser possível que essa asserção faça sentido ou não futuramente.

#Considerações
O presente case foi um esforço de apresentar como o período de Trial pode ou não influenciar na quantidade de assinaturas da plataforma. Constatou-se que a utilização das diversas funcionalidades da plataforma pode influenciar neste processo. Vale refletir também sobre a possibilidade de realizar uma análise que contemple a localização de cada clínica. Se incorporassemos dados secundários como níveis de renda, distribuição demográfica por faixa etária e condições sanitárias, essa abordagem permitiria ou não explorar potenciais padrões de cancelamento em regiões socioeconomicamente mais vulneráveis.\
Priorizei realizar o case para as clinicas `Independentes` por serem, neste banco de dados, um potencial cliente. \
Talvez, tenha terminado com mais questionamentos do que respostas definitivas daquilo que apresento. \
Agradeço muito a leitura deste case e espero novos contatos se interessar. \
Muito obrigado!